## Часть 1
Напишите скрипт, который скачивает все данные с выборов губернатора СПб 2019г для всех избирательных участков. 

Входная точка [по ссылке](http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [1]:
import requests

import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def get_elections_data(to_file=False):
    html = requests.get("http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222")
    if html.status_code != 200:
        raise requests.RequestException

    html = html.content.strip()
    soup = BeautifulSoup(html, "html.parser")
    
    tiks = soup.find_all("a")[8:-3]    
    tik_urls = {}
    base_url = "http://notelections.online"
    
    # получаем URL тиков
    for tik in tiks:
        tik_name = tik.contents[0]
        tik_url = f"{base_url}{tik.get('href')}"
        tik_urls[tik_name] = tik_url

    data = []

    for tik_name, tik_url in tik_urls.items():
        if tik_name == "Цифровые избирательные участки":
            break
            
        # получаем данные с ТИКа
        html = requests.get(tik_url)
        if html.status_code != 200:
            raise requests.RequestException
            
        html = html.content.strip()
        soup = BeautifulSoup(html, "html.parser")
        
        tds = soup.find_all("td")[19:]
        
        # получаем поля
        fields = []
        for td in tds[1:32:3]:
            nobr = td.find_all("nobr")[0]
            fields.append(nobr.contents[0])

        # получаем кандидатов
        candidates = []
        for td in tds[36:44:3]:
            nobr = td.find_all("nobr")[0]
            candidates.append(nobr.contents[0])

        # получаем номера УИК
        idx = 45
        uiks = []
        while True:
            nobr = tds[idx].find_all("nobr")[0]
            a = nobr.find_all("a")
            
            if len(a) == 1:
                uiks.append(a[0].contents[0])
                idx += 1
            else:
                break

        uiks_data = [{} for _ in range(len(uiks))]
        field_idx = 0

        # получаем данные с УИК
        while field_idx < len(fields):
            for i in range(len(uiks)):
                nobrs = tds[idx+i].find_all("nobr")[0].b
                uiks_data[i][fields[field_idx]] = int(nobrs.contents[0])
                
            idx += len(uiks)
            field_idx += 1

        idx += 2
        candidate_idx = 0
        candidates_data = [[0 for _ in range(len(candidates))] 
                           for _ in range(len(uiks))]

        # получаем данные по кандидатам с УИК
        while candidate_idx < len(candidates):
            for i in range(len(uiks)):
                nobrs = tds[idx + i].find_all("nobr")[0].b
                candidates_data[i][candidate_idx] = int(nobrs.contents[0])
                
            idx += len(uiks)
            candidate_idx += 1

        for i in range(len(uiks)):
            data.append({"ТИК №": tik_name,
                         "УИК №": uiks[i]
                        })
            
            data[-1].update({fields[j]: uiks_data[i][fields[j]]
                             for j in range(len(fields))
                            })
            
            data[-1].update({candidates[j]: candidates_data[i][j]
                             for j in range(len(candidates))})
            
    df = pd.DataFrame(data)
    
    if to_file:
        df.to_csv(to_file)
    else:
        return df


In [3]:
get_elections_data()

,ТИК №,УИК №,"Число избирателей, внесенных в список избирателей на момент окончания голосования","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",Число погашенных избирательных бюллетеней,"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования","Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Амосов Михаил Иванович,Беглов Александр Дмитриевич,Тихонова Надежда Геннадьевна
0,Территориальная избирательная комиссия №1,УИК №1,1803,1200,587,11,602,11,587,23,575,0,0,110,345,120
1,Территориальная избирательная комиссия №1,УИК №2,1466,1100,433,14,653,14,430,19,425,0,0,53,326,46
2,Территориальная избирательная комиссия №1,УИК №3,2092,1600,576,22,1002,22,576,15,583,0,0,155,332,96
3,Территориальная избирательная комиссия №1,УИК №4,1056,1000,318,4,678,4,318,13,309,0,0,67,171,71
4,Территориальная избирательная комиссия №1,УИК №5,1827,1400,495,8,897,8,493,16,485,0,0,137,266,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,Территориальная избирательная комиссия №30,УИК №2350,284,300,166,53,81,53,166,0,219,0,0,21,184,14
2017,Территориальная избирательная комиссия №30,УИК №2351,132,150,86,27,37,27,86,2,111,0,0,7,91,13
2018,Территориальная избирательная комиссия №30,УИК №2352,320,350,217,23,110,23,217,4,236,0,0,20,193,23
2019,Территориальная избирательная комиссия №30,УИК №2355,485,500,335,12,153,12,335,2,345,0,0,23,290,32


In [4]:
get_elections_data(to_file="../data/election_data.csv")